# Minimal database interaction example using PyMongo

## Load credentials and sensitive data

In [1]:
import json
# Opening JSON file
with open('../credentials/credentials.json', 'r') as openfile:
    # Reading from json file
    credentials = json.load(openfile)

# Configure the connection to the database and set the collections

In [2]:
import pymongo
import sys
from pymongo import MongoClient
import pandas as pd

In [3]:
db = MongoClient(credentials['pymongo_url'], 
                    credentials['pymongo_port'], 
                     username=credentials['pymongo_username'],
                     password=credentials['pymongo_password']).get_database(credentials['pymongo_database_name'])

In [4]:
current_collection = eval(credentials['meter_current_name'])
power_collection = eval(credentials['meter_power_name'])
voltage_collection = eval(credentials['meter_voltage_name'])

# Query to select dates to calculate the circuit results

In [5]:
import datetime
import ipywidgets as widgets
from ipywidgets import Layout, Box, Dropdown, Label


# available dates in collection:
power_collection_dates=sorted(power_collection.distinct("date"), key=lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
power_collection_meters=sorted(power_collection.distinct("id"))

start_date_selection= widgets.Select(
                                        options= power_collection_dates,
                                        value='2021-10-25',
                                        # rows=10,
                                        description='start date:',
                                        disabled=False
                                    )

display(start_date_selection)

end_date_selection= widgets.Select(
                                        options= power_collection_dates,
                                        value='2021-10-31',
                                        # rows=10,
                                        description='end date:',
                                        disabled=False
                                    )

display(end_date_selection)

Select(description='start date:', index=81, options=('2021-08-05', '2021-08-06', '2021-08-07', '2021-08-08', '…

Select(description='end date:', index=87, options=('2021-08-05', '2021-08-06', '2021-08-07', '2021-08-08', '20…

## Get data for the selected timeframe

In [6]:
user_requested_timestamps= pd.date_range(start_date_selection.value, end_date_selection.value, freq='5Min').tolist()
user_query = { "date": { "$gte": start_date_selection.value , "$lte": end_date_selection.value}}
user_query_power_df = pd.DataFrame(power_collection.find(user_query))

# Get the searchable data for the known times

In [7]:
power_df = pd.DataFrame(power_collection.find())

## Process the retrieved data

In [8]:
def process_retrieved_data(user_query_power_df):
    db_columns=list(user_query_power_df.columns.values)
    time_format = "%H:%M:%S"
    db_times=[]
    db_colums_withoutTime=[]
    for col in db_columns:
        try:
            db_times.append(datetime.datetime.strptime(col, time_format).time())
            # print(f"{col} is the correct date string format.")
        except ValueError:
            db_colums_withoutTime.append(col)
            # print(f"{i} is the incorrect date string format.")
    
    header=[times.strftime("%H:%M:%S") for times in db_times]
    # header.insert(0,'date')
    
    melted_df=pd.melt(user_query_power_df, id_vars=['date', 'id', 'power_type'], value_vars=header, var_name='time', value_name='power')
    # user_query_power_df[user_query_power_df['power_type']=='positive_active'][header]
    pivoted_df=pd.pivot_table(melted_df, values='power', columns='power_type', index=['date', 'time', 'id'])\
                .reset_index()
    pivoted_df.columns.name=None
    pivoted_df.insert(loc=0, column='datetime', value=pd.to_datetime(pivoted_df['date'] + ' ' + pivoted_df['time'], format="%Y-%m-%d %H:%M:%S"))
    processed_power_df=pivoted_df.dropna()
    return processed_power_df

In [9]:
processed_power_df=process_retrieved_data(user_query_power_df)
processed_power_df

,datetime,date,time,id,apparent,positive_active,positive_reactive
0,2021-10-25 00:00:00,2021-10-25,00:00:00,W2,1.542820,0.863157,-1.130695
1,2021-10-25 00:00:00,2021-10-25,00:00:00,W3,4.167453,0.885610,4.037296
2,2021-10-25 00:00:00,2021-10-25,00:00:00,W4,39.954579,38.189706,-10.104962
3,2021-10-25 00:00:00,2021-10-25,00:00:00,W5,0.934930,0.020153,-0.918053
4,2021-10-25 00:00:00,2021-10-25,00:00:00,W6,0.584307,0.042963,-0.048518
...,...,...,...,...,...,...,...
10075,2021-10-31 23:55:00,2021-10-31,23:55:00,W2,1.534457,0.860742,-1.132833
10076,2021-10-31 23:55:00,2021-10-31,23:55:00,W3,4.136900,0.910842,4.000568
10077,2021-10-31 23:55:00,2021-10-31,23:55:00,W4,47.332395,46.345882,-8.114340
10078,2021-10-31 23:55:00,2021-10-31,23:55:00,W5,0.932285,0.019906,-0.917448


In [10]:
processed_power_df_all=process_retrieved_data(power_df)
processed_power_df_all

,datetime,date,time,id,apparent,positive_active,positive_reactive
0,2021-08-05 17:05:00,2021-08-05,17:05:00,W4,96.237015,94.512810,16.111809
1,2021-08-05 17:10:00,2021-08-05,17:10:00,W4,100.207120,98.264920,17.750221
2,2021-08-05 17:15:00,2021-08-05,17:15:00,W4,99.499330,97.550900,17.814768
3,2021-08-05 17:20:00,2021-08-05,17:20:00,W4,99.456580,97.649500,17.126760
4,2021-08-05 17:25:00,2021-08-05,17:25:00,W4,99.666370,97.777120,17.642075
...,...,...,...,...,...,...,...
1038612,2023-10-14 23:50:00,2023-10-14,23:50:00,W4,42.154444,40.080870,-11.502176
1038614,2023-10-14 23:50:00,2023-10-14,23:50:00,W6,0.604524,0.034953,-0.045000
1038615,2023-10-14 23:55:00,2023-10-14,23:55:00,W2,1.474000,0.835000,-1.065263
1038616,2023-10-14 23:55:00,2023-10-14,23:55:00,W4,39.928000,37.724927,-11.560407


# Retrieve predictions

## Filter data corresponding to one of the measurement devices

In [11]:
procesed_power_df_meas_w4=processed_power_df.query('id=="W4"')
procesed_power_df_meas_w4

,datetime,date,time,id,apparent,positive_active,positive_reactive
2,2021-10-25 00:00:00,2021-10-25,00:00:00,W4,39.954579,38.189706,-10.104962
7,2021-10-25 00:05:00,2021-10-25,00:05:00,W4,38.884090,37.048807,-10.205871
12,2021-10-25 00:10:00,2021-10-25,00:10:00,W4,39.680958,37.703280,-10.745584
17,2021-10-25 00:15:00,2021-10-25,00:15:00,W4,39.332112,37.507837,-10.037663
22,2021-10-25 00:20:00,2021-10-25,00:20:00,W4,38.920400,37.015674,-10.195114
...,...,...,...,...,...,...,...
10057,2021-10-31 23:35:00,2021-10-31,23:35:00,W4,44.645709,43.447723,-9.009852
10062,2021-10-31 23:40:00,2021-10-31,23:40:00,W4,42.539431,41.336750,-8.583682
10067,2021-10-31 23:45:00,2021-10-31,23:45:00,W4,48.896885,48.074172,-7.126228
10072,2021-10-31 23:50:00,2021-10-31,23:50:00,W4,48.313760,47.271138,-8.438270


In [12]:
procesed_power_df_meas_w4_all=processed_power_df_all.query('id=="W4"')
procesed_power_df_meas_w4_all

,datetime,date,time,id,apparent,positive_active,positive_reactive
0,2021-08-05 17:05:00,2021-08-05,17:05:00,W4,96.237015,94.512810,16.111809
1,2021-08-05 17:10:00,2021-08-05,17:10:00,W4,100.207120,98.264920,17.750221
2,2021-08-05 17:15:00,2021-08-05,17:15:00,W4,99.499330,97.550900,17.814768
3,2021-08-05 17:20:00,2021-08-05,17:20:00,W4,99.456580,97.649500,17.126760
4,2021-08-05 17:25:00,2021-08-05,17:25:00,W4,99.666370,97.777120,17.642075
...,...,...,...,...,...,...,...
1038600,2023-10-14 23:35:00,2023-10-14,23:35:00,W4,42.080547,40.168370,-10.814491
1038604,2023-10-14 23:40:00,2023-10-14,23:40:00,W4,41.467158,39.551423,-10.641790
1038608,2023-10-14 23:45:00,2023-10-14,23:45:00,W4,41.107518,39.060571,-11.090798
1038612,2023-10-14 23:50:00,2023-10-14,23:50:00,W4,42.154444,40.080870,-11.502176


In [13]:
powerSeries_w4=procesed_power_df_meas_w4[['datetime', 'positive_active']].set_index('datetime')
powerSeries15min_w4=powerSeries_w4.resample('15T').mean()
powerSeries15min_w4

,positive_active
datetime,
2021-10-25 00:00:00,37.647264
2021-10-25 00:15:00,37.335272
2021-10-25 00:30:00,36.781321
2021-10-25 00:45:00,36.311347
2021-10-25 01:00:00,36.821237
...,...
2021-10-31 22:45:00,46.027469
2021-10-31 23:00:00,44.334656
2021-10-31 23:15:00,44.344184


In [14]:
import requests
from io import StringIO
from requests_toolbelt.multipart.encoder import MultipartEncoder

# minimum length required for the input time series
lookback_window = 672

def predict(series, timesteps_ahead, device, power_type):
    """
    Function that returns the prediction of the chosen model using the chosen time series.

    Parameters
    ----------
    series
        The pandas.DataFrame to be used for prediction. It must follow the format shown above. 
        Also, it must have at least lookback window timesteps, as the models need to see that
        far back into the past in order to make a prediction. In this case, it must be at least 7 days
        long (672 timesteps, as we use a resolution of 15 minutes)
    timesteps_ahead
        The amount of timesteps ahead to be redicted. Prediction starts from the next timestep after the
        last one of series. So, as we will perform day ahead forecasting, we need timesteps_ahead=96
    port
        The port the model runs on. 4 models have been trained, and their ports are:
            - 3041 for lgbm w4 positive reactive
            - 3040 for lgbm w4 positive active
            - 3045 for lgbm w6 positive active
            - 3042 for lgbm w6 positive reactive
    Returns
    -------
    pandas.DataFrame
        The result of the prediction
    """
    port_sel={}
    port_sel['W4','active']=3040
    port_sel['W4','reactive']=3041
    port_sel['W6','active']=3045
    port_sel['W6','reactive']=3042
    port=port_sel[device,power_type]
    
    #print("Input dataframe:", series)

    #print ("Port : ", port)
    
    # check length
    if len(series) < lookback_window:
        print("Please input a larger historical time series")
        return

    series = series.to_json()
    input_example = MultipartEncoder(
    fields={"n": str(timesteps_ahead), #96 timesteps for 15min resolution = day ahead forecast
            "series": series,
           }
    )

    result = requests.post(
        f"http://131.154.97.48:{port}/predict", data=input_example, headers={"Content-Type": input_example.content_type}
    ).text

    df = pd.read_json(StringIO(result))
    return df

In [15]:
df_w4 = predict(powerSeries15min_w4, 96, 'W4','active')
df_w4

,positive_active
2021-11-01 00:00:00,43.626579
2021-11-01 00:15:00,42.706357
2021-11-01 00:30:00,39.670648
2021-11-01 00:45:00,38.888780
2021-11-01 01:00:00,40.577193
...,...
2021-11-01 22:45:00,39.078259
2021-11-01 23:00:00,38.767154
2021-11-01 23:15:00,38.767154
2021-11-01 23:30:00,38.612695


In [16]:
val1=df_w4['positive_active'].iloc[0]
print (val1)
vals=[val1*.9, val1*1.1]
vals.sort()
print(vals)
powerCandidates1=procesed_power_df_meas_w4_all[procesed_power_df_meas_w4_all['positive_active'].between(vals[0], vals[1])]
powerCandidates1

43.6265788511
[39.26392096599, 47.98923673621]


,datetime,date,time,id,apparent,positive_active,positive_reactive
99,2021-08-06 01:20:00,2021-08-06,01:20:00,W4,48.557364,47.381023,-8.337425
100,2021-08-06 01:25:00,2021-08-06,01:25:00,W4,48.819864,47.585854,-8.743573
103,2021-08-06 01:40:00,2021-08-06,01:40:00,W4,48.530240,47.114044,-9.512232
104,2021-08-06 01:45:00,2021-08-06,01:45:00,W4,48.461510,47.179229,-8.826363
105,2021-08-06 01:50:00,2021-08-06,01:50:00,W4,48.458205,47.411854,-7.796565
...,...,...,...,...,...,...,...
1038584,2023-10-14 23:15:00,2023-10-14,23:15:00,W4,42.766679,40.847419,-11.043116
1038588,2023-10-14 23:20:00,2023-10-14,23:20:00,W4,42.254214,40.340873,-10.795223
1038600,2023-10-14 23:35:00,2023-10-14,23:35:00,W4,42.080547,40.168370,-10.814491
1038604,2023-10-14 23:40:00,2023-10-14,23:40:00,W4,41.467158,39.551423,-10.641790


In [17]:
procesed_power_df_meas_w6=processed_power_df.query('id=="W6"')

In [18]:
filter1=powerCandidates1['datetime']
procesed_power_df_meas_w6_all=processed_power_df_all.query('id=="W6"')
procesed_power_df_meas_w6_all

,datetime,date,time,id,apparent,positive_active,positive_reactive
197,2021-08-06 09:05:00,2021-08-06,09:05:00,W6,58.080601,-48.680447,30.972766
202,2021-08-06 09:10:00,2021-08-06,09:10:00,W6,59.964800,-50.504429,31.726493
207,2021-08-06 09:15:00,2021-08-06,09:15:00,W6,62.355289,-52.843675,32.489051
212,2021-08-06 09:20:00,2021-08-06,09:20:00,W6,65.589888,-56.022066,33.549350
217,2021-08-06 09:25:00,2021-08-06,09:25:00,W6,68.763551,-59.146108,34.520730
...,...,...,...,...,...,...,...
1038598,2023-10-14 23:30:00,2023-10-14,23:30:00,W6,0.605104,0.035000,-0.045000
1038602,2023-10-14 23:35:00,2023-10-14,23:35:00,W6,0.605000,0.034943,-0.045000
1038606,2023-10-14 23:40:00,2023-10-14,23:40:00,W6,0.603778,0.035056,-0.045000
1038614,2023-10-14 23:50:00,2023-10-14,23:50:00,W6,0.604524,0.034953,-0.045000


In [19]:
filtered_power_w6_all=procesed_power_df_meas_w6_all.loc[procesed_power_df_meas_w6_all['datetime'].isin(filter1)]
filtered_power_w6_all

,datetime,date,time,id,apparent,positive_active,positive_reactive
1207,2021-08-07 01:55:00,2021-08-07,01:55:00,W6,0.594125,0.041515,-0.049412
1212,2021-08-07 02:00:00,2021-08-07,02:00:00,W6,0.591735,0.041665,-0.049340
1232,2021-08-07 02:20:00,2021-08-07,02:20:00,W6,0.604210,0.042206,-0.050343
1242,2021-08-07 02:30:00,2021-08-07,02:30:00,W6,0.604505,0.042378,-0.050235
1247,2021-08-07 02:35:00,2021-08-07,02:35:00,W6,0.606026,0.042426,-0.050385
...,...,...,...,...,...,...,...
1038586,2023-10-14 23:15:00,2023-10-14,23:15:00,W6,0.604960,0.035000,-0.045000
1038590,2023-10-14 23:20:00,2023-10-14,23:20:00,W6,0.604922,0.035000,-0.045000
1038602,2023-10-14 23:35:00,2023-10-14,23:35:00,W6,0.605000,0.034943,-0.045000
1038606,2023-10-14 23:40:00,2023-10-14,23:40:00,W6,0.603778,0.035056,-0.045000


In [20]:
powerSeries_w6=procesed_power_df_meas_w6[['datetime', 'positive_active']].set_index('datetime')
powerSeries15min_w6=powerSeries_w6.resample('15T').mean()
powerSeries15min_w6

,positive_active
datetime,
2021-10-25 00:00:00,0.043272
2021-10-25 00:15:00,0.043444
2021-10-25 00:30:00,0.043085
2021-10-25 00:45:00,0.042837
2021-10-25 01:00:00,0.042941
...,...
2021-10-31 22:45:00,0.043144
2021-10-31 23:00:00,0.043136
2021-10-31 23:15:00,0.043308


In [21]:
df_w6 = predict(powerSeries15min_w6, 96, 'W6','active')
df_w6

,positive_active
2021-11-01 00:00:00,0.096070
2021-11-01 00:15:00,0.096070
2021-11-01 00:30:00,0.149376
2021-11-01 00:45:00,0.172822
2021-11-01 01:00:00,0.207271
...,...
2021-11-01 22:45:00,-26.753191
2021-11-01 23:00:00,-26.753191
2021-11-01 23:15:00,-26.753191
2021-11-01 23:30:00,-26.699885


In [22]:
val2=df_w6['positive_active'].iloc[0]
vals=[val2*.9, val2*1.1]
vals.sort()
print(vals)
powerCandidates2=filtered_power_w6_all[filtered_power_w6_all['positive_active'].between(vals[0], vals[1])]
powerCandidates2

[0.08646310737000001, 0.10567713123000001]


,datetime,date,time,id,apparent,positive_active,positive_reactive
419842,2022-06-02 06:30:00,2022-06-02,06:30:00,W6,17.210604,0.098804,15.357297
465858,2022-07-04 05:25:00,2022-07-04,05:25:00,W6,0.960598,0.097530,0.184667
541656,2022-08-28 18:20:00,2022-08-28,18:20:00,W6,18.222261,0.094269,16.521296
583333,2022-09-26 19:10:00,2022-09-26,19:10:00,W6,0.624500,0.089231,-0.069767
587580,2022-09-29 17:55:00,2022-09-29,17:55:00,W6,0.628817,0.089467,-0.070385
624271,2022-10-25 07:20:00,2022-10-25,07:20:00,W6,0.619300,0.102885,-0.080185
639222,2022-11-04 17:05:00,2022-11-04,17:05:00,W6,0.607642,0.087632,-0.057368
712162,2022-12-25 16:00:00,2022-12-25,16:00:00,W6,17.751957,0.095547,15.646042
842962,2023-04-07 19:50:00,2023-04-07,19:50:00,W6,1.768539,0.101585,1.267179
847395,2023-04-11 19:20:00,2023-04-11,19:20:00,W6,19.181082,0.087649,17.640877


In [23]:
filter2=powerCandidates2['datetime']
procesed_power_df_meas_w3=processed_power_df_all.query('id=="W3"')
filtered_power_w3=procesed_power_df_meas_w3.loc[procesed_power_df_meas_w3['datetime'].isin(filter2)]
filtered_power_w3

,datetime,date,time,id,apparent,positive_active,positive_reactive
419839,2022-06-02 06:30:00,2022-06-02,06:30:00,W3,4.061117,0.905536,3.923305
465855,2022-07-04 05:25:00,2022-07-04,05:25:00,W3,4.155840,0.879161,4.032139
541653,2022-08-28 18:20:00,2022-08-28,18:20:00,W3,4.094028,0.915000,3.954194
583330,2022-09-26 19:10:00,2022-09-26,19:10:00,W3,4.106196,0.914923,3.969318
587577,2022-09-29 17:55:00,2022-09-29,17:55:00,W3,4.082655,0.875758,3.956770
624268,2022-10-25 07:20:00,2022-10-25,07:20:00,W3,4.181613,0.894731,4.048029
639219,2022-11-04 17:05:00,2022-11-04,17:05:00,W3,4.141745,0.885000,4.013786
712159,2022-12-25 16:00:00,2022-12-25,16:00:00,W3,4.025870,0.904940,3.886000


In [24]:
print("The W3 values are active {} and reactive {}".format(filtered_power_w3['positive_active'].mean(), filtered_power_w3['positive_reactive'].mean()))

The W3 values are active 0.8968810487500001 and reactive 3.9729425999999997


In [25]:
procesed_power_df_meas_w5=processed_power_df_all.query('id=="W5"')
procesed_power_df_meas_w5

,datetime,date,time,id,apparent,positive_active,positive_reactive
196,2021-08-06 09:05:00,2021-08-06,09:05:00,W5,0.926915,0.019609,-0.917971
201,2021-08-06 09:10:00,2021-08-06,09:10:00,W5,0.934518,0.019683,-0.924599
206,2021-08-06 09:15:00,2021-08-06,09:15:00,W5,0.934789,0.019692,-0.924982
211,2021-08-06 09:20:00,2021-08-06,09:20:00,W5,0.935088,0.019684,-0.925422
216,2021-08-06 09:25:00,2021-08-06,09:25:00,W5,0.932938,0.019692,-0.923233
...,...,...,...,...,...,...,...
1032883,2023-10-10 00:00:00,2023-10-10,00:00:00,W5,0.941538,0.020000,-0.925072
1034028,2023-10-11 00:00:00,2023-10-11,00:00:00,W5,0.916190,0.020000,-0.895263
1035180,2023-10-12 00:00:00,2023-10-12,00:00:00,W5,0.946538,0.020000,-0.929167
1036330,2023-10-13 00:00:00,2023-10-13,00:00:00,W5,0.917353,0.020000,-0.899344


In [26]:
filtered_power_w5=procesed_power_df_meas_w5.loc[procesed_power_df_meas_w5['datetime'].isin(filter2)]
filtered_power_w5

,datetime,date,time,id,apparent,positive_active,positive_reactive
419841,2022-06-02 06:30:00,2022-06-02,06:30:00,W5,0.914280,0.020359,-0.901900
465857,2022-07-04 05:25:00,2022-07-04,05:25:00,W5,0.925651,0.020190,-0.912702


In [27]:
print("The W5 values are active {} and reactive {}".format(filtered_power_w5['positive_active'].mean(), filtered_power_w5['positive_reactive'].mean()))

The W5 values are active 0.020274786912558324 and reactive -0.9073011224381408
